In [116]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import sqlite3

In [117]:
FLAGS = None

In [118]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where Aktivitetstyp = "train" limit 4000')
for row in rows:
        result.append(row)

rows = np.array(result)
print (rows)

conn.close()

[['Matfettsblandning havssaltat fett 80% berikad typ Bregott' '5' '711.5'
  ..., 'Hård matfettsblandning' '14' 'train']
 ['Matfettsblandning fett 60 % berikad typ Bregott mellan' '6' '534.2' ...,
  'Hård matfettsblandning' '14' 'train']
 ['Bordsmargarin fett 70% berikad typ Milda' '7' '621.0' ...,
  'Hård matfettsblandning' '14' 'train']
 ..., 
 ['Hälleflundra odlad Atlanten rå' '5926' '112.2' ...,
  'Fisk färsk fryst kokt' '6' 'train']
 ['Torsk odlad rå' '5936' '85.7' ..., 'Fisk färsk fryst kokt' '6' 'train']
 ['Tortilla wrap' '5974' '313.7' ..., 'Mjukt bröd' '12' 'train']]


In [119]:
cols = [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38,
        40, 41, 42, 43, 44, 45, 46, 47, 48, 50,
       51, 52, 53, 54, 55, 56, 57, 59]
ndim = len(cols)
labels = np.array(rows.T[61],dtype=int)
print (labels)
#Kolumner med tomma celler: 15, 34, 36, 39, 49, 58, 60
#Ta även bort 0 och 1, 61, 62...
rows = rows[:,cols] #Fancy indexing...
rows

[14 14 14 14 14 14 14 14 14 14  4  4  4 14  4  4  4  4  4  4  4  4  4  4  4
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 18 18
 18 18 16 16 16 16 16 16 16  3 16 16  3 16  5  5 16 16 10 18 16 16 16 16 16
 18 16 16 16 16 16 18 18  8 10 10  8 10  1  1  1  1  9  1  1  1  1  9  9  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  9  1  1  1  1  8
  1  1  9  1  1  9  9  9  1  1  1  1  9  1  1  8  1  9 10 10  1  1  1  1  1
  1  8  8  9  9  9  8  8  1  1  1  8  1 10 10  1  8  8  8  8  8  8  1  1  1
  4  1 10  1  1  8  8  1  4  8  4  4 10  1  8  8  8  1  1  8 10  8  8 10 10
 10 10 10  1  8  1  1 10  1 10 10 10 10  1  4  3 10 10 10  3  3  3 13 13 13
  8  8  8 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13  4 19 11  1 11 11 11 11 11 11 11 11 11 11  5 11 11 11 11
 11 11 11 11 11 11 11  8 11 11 11  8 19 19 19 10  9  9  9  9  9  9  9  9  9
  9  9  9  9 10  3 10 10  8  8 10 10  9  8  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0

array([['711.5', '2977.0', '0.5', ..., '0.0', '0.1', '3.0'],
       ['534.2', '2235.3', '0.5', ..., '0.0', '0', '77.0'],
       ['621.0', '2598.5', '0.3', ..., '0.0', '0', '35.0'],
       ..., 
       ['112.2', '469.6', '0.0', ..., '30.0', '0.44', '52.0'],
       ['85.7', '358.5', '0.0', ..., '30.0', '0.54', '52.0'],
       ['313.7', '1312.6', '57.5', ..., '2.0', '0.6', '158.0']],
      dtype='<U75')

In [120]:
ncat = 20

In [121]:
# Create the model



x = tf.placeholder(tf.float32, [None, ndim])
W = tf.Variable(tf.zeros([ndim, ncat]))
b = tf.Variable(tf.zeros([ncat]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [122]:
y_ = tf.placeholder(tf.float32, [None, ncat])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [123]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [124]:
sess = tf.InteractiveSession()

In [125]:
tf.global_variables_initializer().run()

In [126]:
print(labels)
n_values = np.max(labels) + 1
labels_one_hot=np.eye(n_values)[labels]
print(labels_one_hot)

[14 14 14 14 14 14 14 14 14 14  4  4  4 14  4  4  4  4  4  4  4  4  4  4  4
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 18 18
 18 18 16 16 16 16 16 16 16  3 16 16  3 16  5  5 16 16 10 18 16 16 16 16 16
 18 16 16 16 16 16 18 18  8 10 10  8 10  1  1  1  1  9  1  1  1  1  9  9  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  9  1  1  1  1  8
  1  1  9  1  1  9  9  9  1  1  1  1  9  1  1  8  1  9 10 10  1  1  1  1  1
  1  8  8  9  9  9  8  8  1  1  1  8  1 10 10  1  8  8  8  8  8  8  1  1  1
  4  1 10  1  1  8  8  1  4  8  4  4 10  1  8  8  8  1  1  8 10  8  8 10 10
 10 10 10  1  8  1  1 10  1 10 10 10 10  1  4  3 10 10 10  3  3  3 13 13 13
  8  8  8 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13  4 19 11  1 11 11 11 11 11 11 11 11 11 11  5 11 11 11 11
 11 11 11 11 11 11 11  8 11 11 11  8 19 19 19 10  9  9  9  9  9  9  9  9  9
  9  9  9  9 10  3 10 10  8  8 10 10  9  8  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0

In [127]:
for _ in range(10):
    sess.run(train_step, feed_dict={x: rows, y_: labels_one_hot})

In [128]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [129]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [115]:
# Det här måste vara fel...
print(sess.run(accuracy, feed_dict={x: rows, y_: labels_one_hot}))

0.107692
